## Ejercicio 1: Generación de datos sintéticos de reclamaciones de clientes

En este ejercicio utilizaremos la API de OpenAI para generar reclamaciones de clientes en formato JSON.
Luego, convertiremos los resultados en un DataFrame y aplicaremos un análisis de sentimiento sencillo

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
API_VERSION = userdata.get('OPENAI_API_VERSION')
AZURE_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')

print(type(OPENAI_API_KEY),OPENAI_API_KEY)
print(type(API_VERSION),API_VERSION)
print(type(AZURE_ENDPOINT),AZURE_ENDPOINT)

In [ ]:
import pandas as pd
import json
import os

##### Establecemos el valor de la API key de OpenAI. En una aplicación real, no podemos exponer esta API key en el código, y la cogeríamos con una variable de entorno mediante un os.getenv()

##### Definimos una función para generar reclamaciones sintéticas
Estamos estableciendo valores como el modelo, la temperatura o el max_tokens. Podemos jugar con ellos para ver cómo cambia la generación

In [ ]:
from openai import AzureOpenAI
import time

# Configura el cliente de Azure OpenAI
client = AzureOpenAI(
            api_key = OPENAI_API_KEY,
            api_version=API_VERSION,
            azure_endpoint= AZURE_ENDPOINT# Este endpoint corresponde a la región france-central
)


def generar_reclamaciones(prompt: str, num_muestras: int = 5) -> list:
    """Genera una lista de reclamaciones sintéticas utilizando Azure OpenAI."""
    respuestas = []
    for i in range(num_muestras):
        completion = client.chat.completions.create(
            model="gpt-4o",  # Reemplaza "deployment-name" por el nombre del despliegue configurado en Azure (por ejemplo, "gpt-35-instant")
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5,
            max_tokens=1000,
        )
        # Extraemos el contenido de la respuesta
        texto = completion.choices[0].message.content.strip()
        respuestas.append(texto)
        time.sleep(1)  # Pausa para evitar problemas de rate limit
    return respuestas


##### Y ahora hacemos prompt engineering. Es importante estructurarlo lo mejor posible, como hemos visto en la parte teórica

In [ ]:
# Definimos el prompt para generar una reclamación en formato JSON.
prompt_reclamacion = (
    "Genera una reclamación sintética de un cliente de un banco que incluya los siguientes campos: \n"
    "- una fecha aleatoria (en formato YYYY-MM-DD) entre el 1 de enero de 2021 y el 31 de diciembre de 2024\n"
    "- para las fechas aleatorias devuelvelas todas diferentes"
    "- producto sobre el que está reclamando (puede ser tarjeta de crédito, préstamo, hipoteca...)\n"
    "- canal de contacto (puede ser teléfono, email o chat, elige uno aleatoriamente)\n"
    "- nivel urgencia (bajo, medio o alto, establecido según la problemática expuesta o aleatoriamente)\n"
    "- descripción (descripción larga y detallada del problema)\n"
    "- Algo de texto extra para que parezca más natural. Puede ser un tono neutro o muy negativo\n"
    "Devuelve la reclamación en formato JSON con las claves: 'fecha', 'producto', 'canal', 'urgencia', 'descripcion'.\n"
    "Escribe únicamente el json de la reclamación, sin ningún preámbulo ni texto extra antes o después."
)

##### Generamos las muestras sintéticas

In [ ]:
import json
import re
import pandas as pd

# Generamos las reclamaciones
reclamaciones = generar_reclamaciones(prompt_reclamacion, num_muestras=5)

print("Reclamaciones sintéticas generadas:\n")
for rec in reclamaciones:
    print(rec)
    print("------")

def extraer_json(texto: str) -> str:
    """
    Extrae el bloque JSON del texto eliminando las marcas Markdown si están presentes.
    """
    # Busca un bloque que empiece con ```json y termine con ```
    patron = r"```json\s*(\{.*?\})\s*```"
    coincidencia = re.search(patron, texto, re.DOTALL)
    if coincidencia:
        return coincidencia.group(1)
    else:
        # Si no encuentra las marcas, asume que el texto ya es JSON puro
        return texto.strip()

# Convertimos cada string JSON en un diccionario y creamos un DataFrame
lista_reclamaciones = []
for rec in reclamaciones:
    rec_limpio = extraer_json(rec)
    try:
        diccionario = json.loads(rec_limpio)
        lista_reclamaciones.append(diccionario)
    except json.JSONDecodeError as e:
        print("Error al decodificar JSON:", e)
        print("Contenido problemático:", rec_limpio)

df_reclamaciones = pd.DataFrame(lista_reclamaciones)

print("\nDataFrame de reclamaciones:")
display(df_reclamaciones)

##### - ¿Observas alguna tendencia o patrón en los datos generados?
##### - ¿Cómo podrían ayudar los metadatos (producto, canal, urgencia) a realizar un análisis de sentimiento?

##### Análisis de sentimiento

Una vez hemos generado un dataset, podemos analizar el sentimiento (positivo, neutral, negativo) de las frases para añadirlo como información en el dataframe. Primero, definimos la función y el prompt engineering

In [ ]:
def analizar_sentimiento_api(texto: str) -> str:
    """
    Analiza el sentimiento del texto proporcionado utilizando Azure OpenAI.

    Args:
        texto (str): Texto a analizar.

    Returns:
        str: Sentimiento identificado ("Positivo", "Neutral" o "Negativo").
    """
    prompt_sentimiento = (
        "Analiza el siguiente texto y determina su sentimiento. "
        "Responde únicamente con una de las siguientes etiquetas: Positivo, Neutral, Negativo.\n\n"
        f"Texto: {texto}"
    )
    completion = client.chat.completions.create(
        model="gpt-4o",  # Reemplaza con el nombre de tu despliegue en Azure (por ejemplo, "gpt-35-instant")
        messages=[{"role": "user", "content": prompt_sentimiento}],
        temperature=0.1,  # Menor aleatoriedad para respuestas consistentes
        max_tokens=20,    # Queremos una respuesta directa, así que limitamos el máximo de tokens
    )
    sentimiento = completion.choices[0].message.content.strip()
    return sentimiento

##### Aplicamos el análisis de sentimiento a cada reclamación utilizando la descripción y almacenamos el resultado en una nueva columna 'sentimiento_api

In [ ]:
sentimientos = []
for desc in df_reclamaciones['descripcion']:
    sentimiento = analizar_sentimiento_api(desc)
    sentimientos.append(sentimiento)
    time.sleep(1)  # Pausa para evitar rate limits

df_reclamaciones['sentimiento_api'] = sentimientos

print("\nDataFrame con análisis de sentimiento vía API:")
print(df_reclamaciones)

##### - ¿Observas alguna tendencia o patrón en los datos generados y en el análisis de sentimiento obtenido?
##### - ¿Qué diferencias encuentras entre este análisis de sentimiento basado en la API y métodos tradicionales (p.ej., TextBlob)?